<a href="https://colab.research.google.com/github/GGPengZX/AI_Test/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pdfplumber -i https://mirrors.bfsu.edu.cn/pypi/web/simple/

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 204.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 585.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 541.6 kB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv
import streamlit as st
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
import pdfplumber


def main():
  load_dotenv()

  # locale.setlocale(locale.LC_ALL, 'zh_CN')
  st.set_page_config(page_title="专属PDF知识库")
  st.header("专属PDF知识库💬")
  # 上传文件
  pdf = st.file_uploader("/content/8941576.PDF", type="pdf")

  # 提取文本
  if pdf is not None:
    text = ""
    with pdfplumber.open(pdf) as pdf_reader:
      for page in pdf_reader.pages:
        text += page.extract_text()

    # 文本分片
    text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=50,
      length_function=len
    )
    chunks = text_splitter.split_text(text)

    # 创建embeddings
    embeddings = OpenAIEmbeddings()
    knowledge_base = FAISS.from_texts(chunks, embeddings)
    user_question = st.text_input("来向我提问吧：")
    if user_question:
      docs = knowledge_base.similarity_search(user_question)

      llm = OpenAI()
      chain = load_qa_chain(llm, chain_type="stuff")
      with get_openai_callback() as cb:
        response = chain.run(input_documents=docs, question=user_question)
        print(cb)

      st.write(response)

if __name__ == '__main__':
    main()
